## Higher Order Functions

- Higher order functions are functions that accept another function as an argument, and return them as result
    - Kind of like a decorator in Python

- Like in Python, Scala treats function as **first-class values**
    - This means that functions can be passed as a value, and returned as a result

- This can give us some really flexible use cases, where we can compose higher order functions that take in functions as an input. 

- Imagine we have these functions below
    ```scala
        def sumDoubles(a: Int, b: Int): Int = 
          def double(a: Int): Int = {
            2*a
          }
          if a > b then 0 else double(a) + sumDoubles(a+1, b)
        
        def sumCubes(a: Int, b: Int): Int = 
          def cube(x: Int): Int = {
            x*x*x
          }
          if a > b then 0 else cube(a) + sumCubes(a+1, b)
        
        def sumFactorials(a: Int, b: Int): Int = 
          def factorial(x: Int): Int = {
            if x == 0 then 1 else x * factorial(x-1)
          }
          if a > b then 0 else factorial(a) + sumFactorials(a+1, b)
    ```

- Thinking through the above, these are all generalisations of a common pattern:
    $$\begin{aligned}
        \sum_{n=a}^{b} f(n)
    \end{aligned}$$

- So instead of implementing 3 different functions, why not just abstract the common pattern out?
    ```scala
        def double(a: Int): Int = {
          2*a
        }
        
        def cube(x: Int): Int = {
          x*x*x
        }

        def factorial(x: Int): Int = {
          if x == 0 then 1 else x * factorial(x-1)
        }

        def sumfunc(func: Int => Int, a: Int, b: Int): Int = {
            if a > b then 0
            else func(a) + sumfunc(func, a+1, b)
        }

        def sumfunc_With2Inputs(func: (Int, Int) => Int, a: Int, b: Int): Int = {
            if a > b then 0
            else func(a) + sumfunc(func, a+1, b)
        }

        def sumfunc_With2Inputs(func: (Int, Int) => Int, a: Int, b: Int): Int = {
            if a > b then 0
            else func(a) + sumfunc(func, a+1, b)
        }
    ```

- The new notation here is `func: Int => Int`, which indicates that `func` accepts a function that maps an Int to another Int. 
    - We have also shown how to implement this with a function that takes 2 inputs

### Anonymous Functions

- For cases when you are passing functions into functions, you will almost certainly end up with many small functions, which causes problems because
    - It pollutes your namespace
    - It is **tedious** to write and maintain

- Is there a way we can pass a function "on-demand" without naming it? Kind of like a lambda function in python?

- Scala lets you do this too!
    ```scala
        (x: Int) => x * x * x // (parameter) => body
        
        // same thing
        def cube(x: Int): Int = {
          x*x*x
        }
    ```

- So in fact, our earlier `sumfunc` can be used in a more concise way!!
    ```scala
        def sumCubes(a: Int, b: Int) = sumfunc(x => x*x*x, a, b)
    ```

### Exercise

- The sum function uses linear recursion. Write a tail-recursive version by
replacing the ???

    ```scala
        def sum(f: Int => Int, a: Int, b: Int): Int =
          def loop(a: Int, acc: Int): Int =
            if ??? then ???
            else loop(???, ???)
          loop(???, ???)
    ```

    ```scala
        def sum(f: Int => Int, a: Int, b: Int): Int =
          def loop(a: Int, acc: Int): Int =
            if a > b then acc
            else loop(a+1, acc+f(a))
          loop(a, 0)
    ```

## Currying

- From the section above, we talked about using lambda functions to compose multiple functions together
    
    ```scala
        def sumfunc(func: Int => Int, a: Int, b: Int): Int = {
          if a > b then 0
          else func(a) + sumfunc(func, a+1, b)
        }

        def sumCubes(a: Int, b: Int) = sumfunc(x => x*x*x, a, b)
    ```

- This style presents some problems
    - `a` and `b` are unused by `sumfunc` (directly, anyway)
    - If we want to chain an arbtrary number of functions together, the syntax becomes unwieldy e.g. 
        ```scala 
            sumfunc(x => y => z => x*2*y*3*z, a, b)
        ```

- Let's do a little separation of concerns; we'll abstract `sumfunc` as a function that return a fuction
    ```scala
        def sumfunc(f: Int => Int): (Int, Int) => Int = {
          def composedFunc(a: Int, b: Int): Int = {
            if a > b then 0
            else f(a) + composedFunc(a+1, b)
          }
          composedFunc
        }
    ```

- By writing it this way, our `sumCubes` function can now be written as:
    ```scala
        def sumCubes = sumfunc(x => x * x * x)
    ```

- Yet, this approach still has problems, because we still need to define an intermediate function `sumCubes`!!
    - This causes the namespace pollution we previously discussed

- In actual fact, we can reduce the `sumfunc` definition even further by using a new syntax that Scala enables
    ```scala
        // takes in a function mapping Int => Int
        // you can use this to create a partial function, without supplying `a` and `b`!
        // i.e. val partialfunc = sum(func1); partialfunc(1,10)

        def sum(f: Int => Int)(a: Int, b: Int): Int = {
          if a > b then 0 else f(a) + sum(f)(a+1, b)
        }
    ```


- This can actually be avoided, if we apply this syntax!
    ```scala
        sumfunc(cube)(1, 10)
    ```

- The main value of this syntax is that it allows you to create the equivalent of Python's `partial` function natively! That is, I could define a partial function, and reuse it!
    ```scala
        def sumFunc(f: Int => Int)(a: Int, b: Int, baseCase: Int): Int = {
          def subFunc(a: Int, b: Int, baseCase: Int): Int = {
            if a > b then baseCase
            else f(a) + subFunc(a+1, b, baseCase)
          }
          subFunc
        }
    ```

- QUIZ: Given the function below, what is the type of sum?
    ```scala
        def sum(f: Int => Int)(a: Int, b: Int): Int = ...
    ```
    - `sum` returns an Int eventually

### Exercise

- Write a `product` function that calculates the product of the values of a function for the points on a given interval

- Write `factorial` in terms of `product`

- Can you write a more general function, which generalizes both `sum`
and `product`?

```scala
    def product(a: Int, b: Int): Int = {
      var res = 1
      for (i <- a to b) {
        res = res * i
      }
      res
    }

    def productRecursive(f: Int => Int, a: Int, b: Int): Int = {
      if a > b then 1
      else f(a) * productRecursive(f, a+1, b)
    }

    def productCurry(f: Int => Int)(a: Int, b: Int): Int = {
      if a > b then 1
      else f(a) * productCurry(f)(a+1, b)
    }

    def mapReduce(valueFunc: Int => Int, combineFunc: (Int, Int) => Int, baseCase: Int)(a: Int, b: Int): Int = {
      def recur(a: Int): Int = {
        if a > b then baseCase
        else combineFunc(valueFunc(a), recur(a+1))
      }
      recur(a)
    }

    def productMapReduce(f: Int => Int)(a: Int, b: Int) = {
      mapReduce(f, _ * _, 1)(a, b)
    }

    def sumMapReduce(f: Int => Int)(a: Int, b: Int) = {
      mapReduce(f, _ + _, 0)(a, b)
    }

    def factorial(x: Int): Int = {
      // product(1, x)
      // productRecursive(x => x, 1, x)
      // productCurry(x => x)(1, x)
      productMapReduce(x => x)(1, 10)
    }

```

## Example of Higher Order Functions: Finding Fixed Point

- A fixed point of a function occurs if $f(x) = x$
    - For example, $\sin(0) = 0$
- For some functions, if we iteratively apply the function to an initial estimate, we will end up locating a fixed point! That is;
    - start with random value $a$
    - $f(a)$
    - $f(f(a))$
    - ...
    - this sequence approaches $x$

- Let's try first with this
  ```scala
    import scala.math.abs

    val tolerance = 0.0001

    def isCloseEnough(guess: Double, nextGuess: Double): Boolean = {
      abs((nextGuess - guess) / guess) < tolerance
    }

    def getNextGuess(candidate: Double)(guess: Double): Double = {
      candidate/guess
    }

    def getNextGuessDamping(candidate: Double)(guess: Double): Double = {
      (guess + candidate/guess) / 2
    }

    def fixedPoint(getNextGuessFunc: Double => Double, isCloseEnough: (Double, Double) => Boolean)(firstGuess: Double): Double = {
      def iterate(guess: Double): Double =
        val nextGuess = getNextGuessFunc(guess)
        println(guess + "||" + firstGuess + "||" + nextGuess)
        if isCloseEnough(guess, nextGuess) then nextGuess
        else iterate(nextGuess)
      iterate(firstGuess)
    }

    @main def sqrt(x: Double) = {
      // val res = fixedPoint(getNextGuess(x), isCloseEnough)(1.0)
      val res = fixedPoint(getNextGuessDamping(x), isCloseEnough)(1.0)
      println(res)
    }
  ```

- This is all sensible, but we have a problem. The `getNextGuess` function for `candidate=2` will cycle between returning 2 and 1 and will never terminate!

- To avoid this problem, we modify our candidate acquisition `getNextGuess` by taking an average of the current guess and the next guess! We'll call this `getNextGuessDamping`

- As you can see, the abiliy to pass functions into other functions can give us the ability to do some remarkable things!